# STEP 00: SITE AND SPECIES DETERMINATION

<span style="color: purple">

In this notebook, code will be used to download occurrences of a species in from GBIF to see where the species resides. This will also help determine which sites to focus on.

</span>

In [ ]:
# load packages

# reproducible file paths
import os # make data directories
from glob import glob # search for file paths
import pathlib # work with file system paths

# gbif packages
# may need to install pygbif
import pygbif.occurrences as occ # deal with occurrence data
import pygbif.species as species # work with species names
from getpass import getpass # sign into GBIF

# unzipping and handling gbif data
import zipfile # work with zipfiles
import time # time related functions

# deal with spatial data
import geopandas as gpd # work with geospatial data

# deal w/ other types of data
import pandas as pd # data analysis and manipulation

# packages for visualizing
import hvplot.pandas # plotting
import hvplot.xarray # visualize arrays
import holoviews as hv # save plots

In [2]:
# make reproducible file paths
data_dir = os.path.join(
    # home directory
    pathlib.Path.home(),

    # eda directory
    'earth-analytics',
    'data',

    # project directory
    'spring-2025-habitat-suitability'
)

# make the directory
os.makedirs(data_dir, exist_ok=True)

In [3]:
# set gbif dir
gbif_dir = os.path.join(data_dir, 'gbif_dir')

In [4]:
# access gbif
reset_credentials = True

# enter gbif username, password, and email
credentials = dict(
    GBIF_USER=(input, 'GBIF username'),
    GBIF_PWD=(getpass, 'GBIF password'),
    GBIF_EMAIL=(input, 'GBIF email')
)

for env_variable, (prompt_func, prompt_text) in credentials.items():
    # delete credential from the environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    
    # ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

In [5]:
# species names
species_name = 'Sequoia sempervirens'

# species info for gbif
species_info = species.name_lookup(species_name,
                                   rank = 'SPECIES')

# grab first result
first_result = species_info['results'][0]

# get species key
species_key = first_result['nubKey']

# check first_result
first_result['species'], species_key

('Sequoia sempervirens', 2683909)

In [6]:
#save species code
species_key = 2683909

In [7]:
# set a file pattern 
gbif_pattern = os.path.join(gbif_dir,
                            '*csv')

# download it once
if not glob(gbif_pattern):
    # submit query to GBIF to get all data from all years
    gbif_query = occ.download([
        f'speciesKey = {species_key}',
        'hasCoordinate = True'
    ])

    # only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]
        download_key = os.environ['GBIF_DOWNLOAD_KEY'] 

        # wait for downlaod to build
        wait = occ.download_meta(download_key)['status']
        while not wait == 'SUCCEEDED' :
            wait = occ.download_meta(download_key)['status']
            time.sleep(5)
    # download the data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'],
        path = data_dir
    )

    # unzip it
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path = gbif_dir)

# find csv file path
gbif_path = glob(gbif_pattern)[0]

<span style="color: purple">

redwood gbif download citation:

GBIF.org (05 March 2025) GBIF Occurrence Download  https://doi.org/10.15468/dl.yh4ygk

</span>

In [8]:
# open gbif data
gbif_df = pd.read_csv(
    gbif_path,
    delimiter = '\t'
)

# check dataframe
gbif_df

C:\Users\riede\AppData\Local\Temp\ipykernel_23820\4019471255.py:2: DtypeWarning: Columns (10,46) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif_df = pd.read_csv(


,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,986394934,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:132704,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,"Deyholos, M. K.",NaN,CC0_1_0,University of Alberta Museums,"Deyholos, M. K.",NaN,NaN,2025-02-27T20:43:10.577Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;OCCURRENCE_STATUS...
1,930742189,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Maurice J. Kaufmann,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:39.109Z,StillImage,NaN
2,930742172,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Maurice J. Kaufmann,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:43.284Z,StillImage,NaN
3,930742166,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Maurice J. Kaufmann,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:42.424Z,StillImage,NaN
4,930741653,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-baskauf/41...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Steven J. Baskauf,2005-07-28T00:00:00,CC0_1_0,NaN,Steven J. Baskauf,NaN,native,2025-02-06T17:33:40.327Z,StillImage,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18309,1024187447,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/777058,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Ken-ichi Ueda,2014-07-10T07:56:43,CC0_1_0,Ken-ichi Ueda,Ken-ichi Ueda,NaN,NaN,2025-02-16T16:00:19.901Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...
18310,1024186865,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/775641,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,Chris Cook,2014-07-09T04:59:15,CC_BY_NC_4_0,Chris Cook,Chris Cook,NaN,NaN,2025-02-16T12:07:51.739Z,NaN,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...
18311,1024184572,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/769678,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,JJ Johnson,2014-07-05T03:18:09,CC_BY_NC_4_0,JJ Johnson,JJ Johnson,NaN,NaN,2025-02-16T12:07:51.506Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...
18312,1024184101,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/767949,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,James Maughn,2014-07-04T06:20:45,CC_BY_NC_4_0,James Maughn,James Maughn,NaN,NaN,2025-02-16T12:07:51.396Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...


In [9]:
# see all gbif_df columns
gbif_df.columns

Index(['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species', 'infraspecificEpithet',
       'taxonRank', 'scientificName', 'verbatimScientificName',
       'verbatimScientificNameAuthorship', 'countryCode', 'locality',
       'stateProvince', 'occurrenceStatus', 'individualCount',
       'publishingOrgKey', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation',
       'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day',
       'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord',
       'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber',
       'identifiedBy', 'dateIdentified', 'license', 'rightsHolder',
       'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted',
       'mediaType', 'issue'],
      dtype='object')

In [10]:
# make gbif_df spatial
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df,
        geometry = gpd.points_from_xy(
            # x value comes from decimalLongitude
            gbif_df.decimalLongitude,
            # y value comes from decimalLatitude
            gbif_df.decimalLatitude
        ),
        # assign a crs to the gdf
        crs = 'EPSG:4326'
    )
)

# check gbif_gdf
gbif_gdf.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,geometry
0,986394934,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:132704,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,NaN,CC0_1_0,University of Alberta Museums,"Deyholos, M. K.",NaN,NaN,2025-02-27T20:43:10.577Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;OCCURRENCE_STATUS...,POINT (-113.51667 53.51667)
1,930742189,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:39.109Z,StillImage,NaN,POINT (-124.0411 41.3232)
2,930742172,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:43.284Z,StillImage,NaN,POINT (-124.0411 41.3232)
3,930742166,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-kaufmannm/...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,1977-01-01T00:00:00,CC0_1_0,NaN,Maurice J. Kaufmann,NaN,native,2025-02-06T17:33:42.424Z,StillImage,NaN,POINT (-124.0411 41.3232)
4,930741653,0096dfc0-9925-47ef-9700-9b77814295f1,http://bioimages.vanderbilt.edu/ind-baskauf/41...,Plantae,Tracheophyta,Pinopsida,Pinales,Cupressaceae,Sequoia,Sequoia sempervirens,...,2005-07-28T00:00:00,CC0_1_0,NaN,Steven J. Baskauf,NaN,native,2025-02-06T17:33:40.327Z,StillImage,NaN,POINT (-124.087 41.79557)


In [13]:
# plot gbif_gdf to see where the occurrences are
gbif_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Coast Redwood (Sequoia sempervirens) Occurrences From GBIF',
    # set fill and line color
    fill_color = None, line_color = 'limegreen',
    # set frame size
    frame_width = 700
)


c:\Users\riede\miniconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

<span style="color: purple">

The forest url and API is from here: https://data-usfs.hub.arcgis.com/datasets/usfs::fs-national-forests-dataset-us-forest-service-proclaimed-forests/about

Try downloading forest boundaries with API from the above site:

</span>

In [14]:
# set up national forest url
forest_url = ("https://apps.fs.usda.gov/arcx/rest/services/EDW"
              "/EDW_ProclaimedForestBoundaries_01"
              "/MapServer/0/query?outFields=*&where=1%3D1&f=geojson")

# set up path to save forest data
forest_dir = os.path.join(data_dir, 'forest-dir')
os.makedirs(forest_dir, exist_ok=True)


# Join forest shapefile path
forest_path = os.path.join(forest_dir, 'S_USA.ProclaimedForestBoundaries.shp')

# Only download once
if not os.path.exists(forest_path):
    forest_gdf = gpd.read_file(forest_url)
    forest_gdf.to_file(forest_path)

# Create forest_gdf
forest_gdf = gpd.read_file(forest_path)

In [15]:
# Check forest_gdf
forest_gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPE.AREA,SHAPE.LEN,geometry
0,200571,66329010328,Bighorn National Forest,1.112646e+06,0.509899,4.943363,"POLYGON ((-107.54298 44.93779, -107.54298 44.9..."
1,200572,93007010328,Gifford Pinchot National Forest,1.532173e+06,0.722290,11.679336,"MULTIPOLYGON (((-122.27148 45.75119, -122.2711..."
2,200573,96812010328,Manti-La Sal National Forest,1.337654e+06,0.561732,9.953308,"MULTIPOLYGON (((-111.40257 39.98081, -111.4009..."
3,200574,96813010328,Uinta National Forest,9.617430e+05,0.411377,6.874785,"MULTIPOLYGON (((-111.55131 40.59378, -111.5513..."
4,200575,105935010328,Kaibab National Forest,1.601003e+06,0.647156,9.140002,"MULTIPOLYGON (((-112.06366 36.8781, -112.06366..."
5,200576,106640010328,Fremont National Forest,1.713917e+06,0.760518,11.282002,"MULTIPOLYGON (((-120.53326 42.57082, -120.5285..."
6,200577,106887010328,Mt. Baker National Forest,1.317677e+06,0.648567,10.659633,"MULTIPOLYGON (((-121.75954 48.99733, -121.7512..."
7,200578,107266010328,Olympic National Forest,6.958685e+05,0.337426,8.179740,"MULTIPOLYGON (((-123.23508 48.01368, -123.2297..."
8,200579,107474010328,Wallowa National Forest,1.064857e+06,0.496222,9.027734,"MULTIPOLYGON (((-117.56261 45.48274, -117.5625..."
9,200580,108201010328,Wenatchee National Forest,1.963052e+06,0.952602,15.707300,"MULTIPOLYGON (((-121.17501 47.19079, -121.1750..."


<span style="color: purple">

For some reason, when I try downloading the forest boundaries with the API, I only get 30 results, even though the [website about the API/Dataset](https://data-usfs.hub.arcgis.com/datasets/usfs::fs-national-forests-dataset-us-forest-service-proclaimed-forests/about) says there are 154 results, including the two forests I want, Los Padres National Forest and Rogue River-Siskiyou National Forest. Those two forests are not part of the 30 results from the API. So, instead of using the API, I downloaded the shapefile of the Dataset as a zipfile and will access it below.

</span>

In [16]:
## Load in the proclaimed forest boundary file from my computer
# writing over the variables I defined above since I won't use the API
# open national_forests_path
forest_path = (
    os.path.join(forest_dir,
                 'FS_National_Forests_Dataset_(US_Forest_Service_Proclaimed_Forests).zip'))
forest_path

# open polygon
forest_gdf = gpd.read_file(forest_path)
forest_gdf


,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPEAREA,SHAPELEN,geometry
0,200571,66329010328,Bighorn National Forest,1112645.655,0.509899,4.943363,"POLYGON ((-107.54297 44.93778, -107.54297 44.9..."
1,200572,93007010328,Gifford Pinchot National Forest,1532172.644,0.722290,11.679336,"MULTIPOLYGON (((-122.27146 45.75118, -122.2711..."
2,200573,96812010328,Manti-La Sal National Forest,1337654.214,0.561732,9.953308,"MULTIPOLYGON (((-111.40255 39.9808, -111.40093..."
3,200574,96813010328,Uinta National Forest,961743.024,0.411377,6.874785,"MULTIPOLYGON (((-111.5513 40.59377, -111.5513 ..."
4,200575,105935010328,Kaibab National Forest,1601002.978,0.647156,9.140002,"MULTIPOLYGON (((-112.06365 36.8781, -112.06365..."
...,...,...,...,...,...,...,...
149,200720,295501010328,Grand Mesa National Forest,351209.089,0.147989,4.076972,"MULTIPOLYGON (((-107.8139 39.37261, -107.8136 ..."
150,200721,295502010328,Arapaho National Forest,767529.673,0.327500,9.982819,"MULTIPOLYGON (((-105.82719 40.24581, -105.8277..."
151,200722,295503010328,Stanislaus National Forest,1092025.607,0.454330,6.241501,"MULTIPOLYGON (((-120.40147 38.30669, -120.4014..."
152,200723,295504010328,Tahoe National Forest,1225633.350,0.518547,10.476720,"MULTIPOLYGON (((-121.14404 39.463, -121.13938 ..."


In [17]:
#check gbif_gdf crs before doing intersection below
gbif_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
#check forest_gdf crs before doing intersection below
forest_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [19]:
# change forest_gdf crs to gbif_gdf crs
forest_gdf = forest_gdf.to_crs(gbif_gdf.crs)
forest_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

<span style="color: purple">

I have commented out the two cells below where I intersect the gbif_gdf and forest_gdf because it takes a few minutes to run. I only needed to run it once to help aid my site selection. When I intersected the two GeoDataFrames, I got these results:

* Los Padres National Forest        346
* Six Rivers National Forest         36
* Siskiyou National Forest           26
* San Bernardino National Forest      7
* Klamath National Forest             3
* Trinity National Forest             3
* Sierra National Forest              2
* Angeles National Forest             2
* Siuslaw National Forest             1
* Mendocino National Forest           1
* Stanislaus National Forest          1

I am leaving the code cells above where I identify the Coordinate Reference Systems and make gbif_gdf and forest_gdf have the same CRS because I think those GDFs having the same CRS could be helpful for future code.

</span>

In [20]:
# intersect coast redwood occurrence with forest_gdf
# only keep the occurrences that are in the forest_gdf shapefile
# gbif_forest_intersection = gpd.overlay(gbif_gdf, forest_gdf, how = 'intersection')

In [21]:
# how many occurrences per site?
# value_counts = gbif_forest_intersection['FORESTNAME'].value_counts()
# value_counts

In [22]:
# Create individual gdf for Siskiyou National Forest
siskiyou_forest_gdf = forest_gdf[forest_gdf['FORESTNAME']=='Siskiyou National Forest']

# Plot Rogue River-Siskiyou National Forest
siskiyou_forest_boundary_plot = siskiyou_forest_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Siskiyou National Forest, Oregon & California, USA',
    # set fill and line color
    fill_color = None, line_color = 'pink',
    frame_width = 475
)
siskiyou_forest_boundary_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [23]:
# Create individual gdf for Los Padres National Forest
padres_forest_gdf = forest_gdf[forest_gdf['FORESTNAME']=='Los Padres National Forest']

# Plot Uinta National Forest
padres_forest_boundary_plot = padres_forest_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = 'Los Padres National Forest, California, USA',
    # set fill and line color
    fill_color = None, line_color = 'blue',
    # set frame width
    frame_width = 475
)
padres_forest_boundary_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [24]:
# combine both sites into one
sites_gdf = gpd.GeoDataFrame(pd.concat([siskiyou_forest_gdf, padres_forest_gdf], ignore_index=True))
sites_gdf

,OBJECTID,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPEAREA,SHAPELEN,geometry
0,200683,295464010328,Siskiyou National Forest,1161692.775,0.513890,6.990039,"MULTIPOLYGON (((-124.02284 42.83323, -124.0209..."
1,200607,295387010328,Los Padres National Forest,1956740.523,0.782005,12.961269,"MULTIPOLYGON (((-120.74838 35.44383, -120.7484..."


In [25]:
# Plot both National Forests
sites_boundaries_plot = sites_gdf.hvplot(
    # treat the plot as geographic and assume lat/lon coordinates
    geo = True,
    # overlay the plot on EsriImagery tiles
    tiles = 'EsriImagery',
    # set title
    title = ('Siskiyou and Los Padres National Forests, Oregon & California, USA'),
    # set fill and line color
    fill_color = None, line_color = 'white',
    # set frame width
    frame_width = 650
)
sites_boundaries_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

<span style="color: purple">

Use holoviews to save the plots and use store magic to save project data directory and site GDFs:

</span>

In [26]:
# Commenting this cell out since the plots are saved.
# Uncomment if the plots are changed and need to be resaved
# hv.save(siskiyou_forest_boundary_plot,
#         'Siskiyou National Forest Boundary, Oregon & California, USA.html')
# hv.save(padres_forest_boundary_plot,
#         'Los Padres National Forest Boundary, California, USA.html')
# hv.save(sites_boundaries_plot,
#         'Siskiyou & Los Padres National Forest Boundaries, Oregon & CA, USA.html')

In [27]:
%store data_dir sites_gdf siskiyou_forest_gdf padres_forest_gdf

Stored 'data_dir' (str)
Stored 'sites_gdf' (GeoDataFrame)
Stored 'siskiyou_forest_gdf' (GeoDataFrame)
Stored 'padres_forest_gdf' (GeoDataFrame)
